In [75]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

# 2016_2c_parcial_1

Contamos con un cluster que tiene 4 computadoras. Queremos
aprovechar el paralelismo del cluster para calcular los números primos
entre 2 y 20.000.000. Para esto usaremos el conocido algoritmo de la
criba de Eratóstenes. Por ejemplo si empezamos con una lista de tipo
(2,3,4,5,6,7,8...) en un primer paso eliminamos todos los que son
mayores a 2 y divisibles por 2 y nos queda (2,3,5,7,9,11,13…) luego
eliminamos todos los mayores a 3 divisibles por 3 y nos queda
(2,3,5,7,11,13….etc) luego todos los divisibles por 5 y así
sucesivamente. El resultado final es una lista de números que son
primos. Programar este programa usando PySpark

In [113]:
def isDivisiblePor(x,y):
    return ((x%y) == 0)

def myPrint(x):
    print(x)

data = range(2,2000000)
rdd = sc.parallelize(data,4)

#rdd.filter(lambda x: (i>=x or (x>i and not isDivisiblePor(x,i))))
pivote = 1

pivote = rdd.reduce(lambda x,y: x if (x > pivote and x < y) else y)
while(pivote != prev):
    rdd = rdd.filter(lambda x: (pivote>=x or (x>pivote and not isDivisiblePor(x,pivote)))).cache()
    
    prev = pivote
    pivote = rdd.reduce(lambda x,y: x if (x > pivote and x < y) else y)
    
    
rdd.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9506.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9506.0 (TID 34352, localhost): java.lang.StackOverflowError
	at java.lang.ReflectiveOperationException.<init>(ReflectiveOperationException.java:89)
	at java.lang.reflect.InvocationTargetException.<init>(InvocationTargetException.java:72)
	at sun.reflect.GeneratedMethodAccessor6.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:606)
	at java.io.ObjectStreamClass.invokeReadObject(ObjectStreamClass.java:1058)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:1907)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:1806)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1353)
